In [1]:
import os
from IPython import get_ipython
ipython = get_ipython()

In [2]:
ipython.magic("load_ext rpy2.ipython")
string_libraries = """R library(hsi); library(raster); library(usethis);
                        library(devtools); library(rgdal); library(future);
                        library(furrr);
                   """
ipython.magic(string_libraries)

R[write to console]: Loading required package: sp

R[write to console]: rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.0.4, released 2020/01/28
Path to GDAL shared files: /usr/share/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 6.3.1, February 10th, 2020, [PJ_VERSION: 631]
Path to PROJ shared files: /usr/share/proj
Linking to sp version:1.4-2
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.

R[write to console]: 
Attaching package: ‘future’


R[write to console]: The following object is masked from ‘package:raster’:

    values




In [2]:
#
ipython.magic("load_ext rpy2.ipython")
#

string_data_input = """R poncaloc <- rgdal::readOGR("/shared_volume/Ponca_DV_loc/","poncadav2");
                         ponca_mask <- raster::raster("/shared_volume/Ponca_DV/poncamask.tif")
                    """
ipython.magic(string_data_input)

#
poncaloc = ipython.magic("Rget poncaloc")
ponca_mask = ipython.magic("Rget ponca_mask")

print(poncaloc)
print(ponca_mask)
#

In [3]:
#
ipython.magic("load_ext rpy2.ipython")
print(poncaloc)
ipython.magic("Rpush poncaloc")
#
string_transform = """R poncaloc <- sp::spTransform(poncaloc,
                                                    CRSobj = "+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0")
                   """
ipython.magic(string_transform)

#
poncaloc = ipython.magic("Rget poncaloc")
print(poncaloc)
#

In [4]:
#
ipython.magic("load_ext rpy2.ipython")
print(poncaloc)
ipython.magic("Rpush poncaloc")
#
string_libraries = """R library(hsi)"""

ipython.magic(string_libraries)

string_test = """R test_sp <- sp_temporal_data(occs=poncaloc,longitude = "coords.x1",
                                               latitude = "coords.x2",sp_year_var="Year",
                                               layers_by_year_dir ="/shared_volume/forest_jEquihua_mar/",
                                               layers_ext = "*.tif$",reclass_year_data = T)
              """
ipython.magic(string_test)

#
test_sp = ipython.magic("Rget test_sp")
print(test_sp)
#

In [5]:
#
ipython.magic("load_ext rpy2.ipython")


string_libraries = """R library(hsi);library(raster)"""

ipython.magic(string_libraries)

print(test_sp)
print(ponca_mask)
ipython.magic("Rpush test_sp")
ipython.magic("Rpush ponca_mask")
#

#Filtrar las localidades que se usaran mediante la mascara
string_filter = """R test_sp_mask <- occs_filter_by_mask(test_sp,ponca_mask)
                """
ipython.magic(string_filter)

#Limpia localidades duplicadas por anio

string_clean_test = """R test_sp_clean <- clean_dup_by_year(this_species = test_sp,
                                                            threshold = res(ponca_mask)[1])
                    """

ipython.magic(string_clean_test)

string_extract = """R e_test <- extract_by_year(this_species=test_sp_clean,layers_pattern="_mar")
                 """

ipython.magic(string_extract)


#
e_test = ipython.magic("Rget e_test")
print(e_test)
#

In [6]:
#
ipython.magic("load_ext rpy2.ipython")
print(e_test)
ipython.magic("Rpush e_test")
#
string_libraries = """R library(hsi)"""

ipython.magic(string_libraries)

string_bestmodel = """R best_model_2004 <- find_best_model(this_species = e_test,
                                                           cor_threshold = 0.8,
                                                           ellipsoid_level = 0.975,
                                                           nvars_to_fit = 3,E = 0.05,
                                                           RandomPercent = 70,
                                                           NoOfIteration = 1000,
                                                           parallel = TRUE,
                                                           n_cores = 24,
                                                           plot3d = FALSE)
                   """

ipython.magic(string_bestmodel)

#
best_model_2004 = ipython.magic("Rget best_model_2004")
print(best_model_2004)
#

In [7]:
#
ipython.magic("load_ext rpy2.ipython")

string_libraries = """R library(hsi);library(raster)"""

ipython.magic(string_libraries)

print(best_model_2004)
print(ponca_mask)
ipython.magic("Rpush best_model_2004")
ipython.magic("Rpush ponca_mask")
#

results = "/shared_volume/new_model_parallel/19_05_2021_2/"

string_temporal_proj = """R temporal_projection(this_species = best_model_2004,
                                                save_dir = "/shared_volume/new_model_parallel/19_05_2021_2/",
                                                sp_mask = ponca_mask,
                                                crs_model = NULL,
                                                sp_name ="pan_onca",
                                                plot3d = FALSE)
                      """

if not os.path.exists(results):
    os.makedirs(results)
    
ipython.magic(string_temporal_proj)

#
temporal_projection = ipython.magic("Rget temporal_projection")
print(temporal_projection)
#